In [3]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
from sklearn.model_selection import GroupKFold
# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [5]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [8]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 
# 'max_bin': 100

## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514
# not our case but good to know cons

In [9]:
########################### Vars
#################################################################################
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1941               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = True               # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
ORIGINAL = '/content/gdrive/My Drive/m5/'
BASE     = '/content/gdrive/My Drive/m5_new/grid_part_1.pkl'
PRICE    = '/content/gdrive/My Drive/m5_new/grid_part_2.pkl'
CALENDAR = '/content/gdrive/My Drive/m5_new/grid_part_3.pkl'
LAGS     = '/content/gdrive/My Drive/m5_new/lags_df_28.pkl'
MEAN_ENC = '/content/gdrive/My Drive/m5_new/mean_encoding_df.pkl'


# AUX(pretrained) Models paths
AUX_MODELS = '/content/gdrive/My Drive/m5_new/'


#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())

#FOLDS
CV_FOLDS = [0,1,2,3]


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [11]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    ## Initiating our GroupKFold
    folds = GroupKFold(n_splits=4)

    # get subgroups for each week, year pair
    grid_df['groups'] = grid_df['tm_w'].astype(str) + '_' + grid_df['tm_y'].astype(str)
    split_groups = grid_df[train_mask]['groups']

    # Main Data
    X,y = grid_df[train_mask][features_columns], grid_df[train_mask][TARGET]
        
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)):
        print('Fold:',fold_)
        print(len(trn_idx),len(val_idx))
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        v_X, v_y   = X.iloc[val_idx,:], y[val_idx] 
        train_data = lgb.Dataset(tr_x, label=tr_y)
        valid_data = lgb.Dataset(v_X, label=v_y)  

        seed_everything(SEED)
        estimator = lgb.train(
                 {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.6,
                    'max_bin': 100,
                    'n_estimators': 1450,
                    'early_stopping_rounds': 30,
                    'boost_from_average': False,
                    'verbose': -1,
                } ,
                train_data,
                valid_sets = [train_data, valid_data],
                verbose_eval = 100
            )
        # Save model - it's not real '.bin' but a pickle file
        # estimator = lgb.Booster(model_file='model.txt')
        # can only predict with the best iteration (or the saving iteration)
        # pickle.dump gives us more flexibility
        # like estimator.predict(TEST, num_iteration=100)
        # num_iteration - number of iteration want to predict with, 
        # NULL or <= 0 means use best iteration
        model_name = 'lgb_model_'+ store_id +'_'+str(fold_)+'.bin'
        pickle.dump(estimator, open(model_name, 'wb'))

        # Remove temporary files and objects 
        # to free some hdd space and ram memory
        del train_data, valid_data, estimator
        gc.collect()

    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
Fold: 0
3596122 1192145
Training until validation scores don't improve for 30 rounds.
[100]	training's rmse: 2.56016	valid_1's rmse: 2.59594
[200]	training's rmse: 2.4615	valid_1's rmse: 2.50445
[300]	training's rmse: 2.42741	valid_1's rmse: 2.48399
[400]	training's rmse: 2.40542	valid_1's rmse: 2.47598
[500]	training's rmse: 2.38839	valid_1's rmse: 2.47056
[600]	training's rmse: 2.37278	valid_1's rmse: 2.46688
[700]	training's rmse: 2.35919	valid_1's rmse: 2.46333
[800]	training's rmse: 2.34717	valid_1's rmse: 2.4616
[900]	training's rmse: 2.33637	valid_1's rmse: 2.46015
[1000]	training's rmse: 2.32556	valid_1's rmse: 2.45802
Early stopping, best iteration is:
[1029]	training's rmse: 2.32233	valid_1's rmse: 2.45775
Fold: 1
3587602 1200665
Training until validation scores don't improve for 30 rounds.
[100]	training's rmse: 2.58892	valid_1's rmse: 2.55975
[200]	training's rmse: 2.48642	valid_1's rmse: 2.46735
[300]	training's rmse: 2.45286	valid_1's rmse: 2.44081
[400]	traini

In [ ]:
lgb_model_

In [12]:
########################### Predict
#################################################################################

for fold_ in [0,1,2,3]:
    print("FOLD:", fold_)
    # Join back the Test dataset with 
    # a small part of the training data 
    # to make recursive features
    all_preds = pd.DataFrame()
    base_test = get_base_test()
    # Timer to measure predictions time 
    main_time = time.time()

    # Loop over each prediction day
    # As rolling lags are the most timeconsuming
    # we will calculate it for whole day
    for PREDICT_DAY in range(1,29):    
        print('Predict | Day:', PREDICT_DAY)
        start_time = time.time()

        # Make temporary grid to calculate rolling lags
        grid_df = base_test.copy()
        grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)

        for store_id in STORES_IDS:
        
            # Read all our models and make predictions
            # for each day/store pairs
            model_path = 'lgb_model_'+store_id+'_'+str(fold_)+'.bin' 
            if USE_AUX:
                model_path = '' + model_path

            estimator = pickle.load(open(model_path, 'rb'))

            day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
            store_mask = base_test['store_id']==store_id

            mask = (day_mask)&(store_mask)
            base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

        # Make good column naming and add 
        # to all_preds DataFrame
        temp_df = base_test[day_mask][['id',TARGET]]
        temp_df.columns = ['id','F'+str(PREDICT_DAY)]
        if 'id' in list(all_preds):
            all_preds = all_preds.merge(temp_df, on=['id'], how='left')
        else:
            all_preds = temp_df.copy()
        all_preds = all_preds.reset_index(drop=True)
        print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                      ' %0.2f min total |' % ((time.time() - main_time) / 60),
                      ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    all_preds.to_csv('all_preds_'+str(fold_)+'.csv',index=False)
    !cp 'all_preds_'+str(fold_)+'.csv' /content/gdrive/'My Drive'/m5_new
    del temp_df, all_preds

FOLD: 0
Predict | Day: 1
##########  0.74 min round |  0.74 min total |  39840.63 day sales |
Predict | Day: 2
##########  0.65 min round |  1.39 min total |  37056.26 day sales |
Predict | Day: 3
##########  0.64 min round |  2.03 min total |  37019.72 day sales |
Predict | Day: 4
##########  0.63 min round |  2.66 min total |  37172.93 day sales |
Predict | Day: 5
##########  0.63 min round |  3.29 min total |  42358.75 day sales |
Predict | Day: 6
##########  0.63 min round |  3.92 min total |  51003.85 day sales |
Predict | Day: 7
##########  0.62 min round |  4.54 min total |  52010.93 day sales |
Predict | Day: 8
##########  0.62 min round |  5.16 min total |  44244.34 day sales |
Predict | Day: 9
##########  0.63 min round |  5.79 min total |  38827.85 day sales |
Predict | Day: 10
##########  0.64 min round |  6.42 min total |  43805.29 day sales |
Predict | Day: 11
##########  0.63 min round |  7.05 min total |  45121.14 day sales |
Predict | Day: 12
##########  0.65 min round

In [ ]:
all_preds_0=pd.read_csv('all_preds_0'+'.csv')
all_preds_1=pd.read_csv('all_preds_1'+'.csv')
all_preds_2=pd.read_csv('all_preds_2'+'.csv')
all_preds_3=pd.read_csv('all_preds_3'+'.csv')

In [ ]:
# Create Dummy DataFrame to store predictions
final_all_preds = pd.DataFrame()
final_all_preds['id'] = all_preds_1['id']
for item in all_preds_1:
    if item!='id':
        final_all_preds[item]=(all_preds_0[item]*(1/4))+(all_preds_1[item]*(1/4))+(all_preds_2[item]*(1/4) + all_preds_3[item]*(1/4))
final_all_preds

In [ ]:
0	HOBBIES_1_001_CA_1_evaluation	0.897567	0.808433	0.787248	0.834224	1.023035	1.240877	1.142573	1.025858	0.869062	0.954067	0.819747	1.053971	1.252962	1.169393	0.899686	0.867141	0.849904	0.841423	0.972701	1.255027	1.143760	0.938854	0.851907	0.850253	0.899863	1.088019	1.209305	1.073520
1	HOBBIES_1_002_CA_1_evaluation	0.213175	0.190870	0.194972	0.192493	0.222992	0.285271	0.312677	0.237396	0.207413	0.242501	0.211367	0.279419	0.354906	0.331790	0.228765	0.229240	0.223464	0.232000	0.270699	0.339222	0.362581	0.243112	0.229265	0.233733	0.248364	0.294227	0.379558	0.405173
2	HOBBIES_1_003_CA_1_evaluation	0.520261	0.471055	0.484993	0.504455	0.672100	0.799509	0.801922	0.553256	0.526301	0.513892	0.486507	0.718935	0.842681	0.777672	0.509570	0.502527	0.497165	0.507420	0.669064	0.811831	0.802531	0.527526	0.471016	0.481420	0.506258	0.684005	0.781834	0.785586
3	HOBBIES_1_004_CA_1_evaluation	1.515989	1.303852	1.299618	1.353299	1.810563	2.557639	2.840314	1.936896	1.364120	1.369339	1.348411	2.176968	2.847142	2.942518	1.648374	1.457491	1.317094	1.419085	1.880304	2.538570	2.938713	1.634545	1.339442	1.322486	1.436190	1.855930	2.566287	2.624366
4	HOBBIES_1_005_CA_1_evaluation	1.114401	0.970305	0.939629	0.999928	1.179970	1.410065	1.534941	1.219148	1.056377	1.141098	1.029266	1.398179	1.646224	1.656603	1.128693	1.025425	1.034846	1.038983	1.244264	1.464190	1.542456	1.105556	1.009824	1.015949	1.015203	1.298450	1.558173	1.432512
...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...	...
30485	FOODS_3_823_WI_3_evaluation	0.500702	0.462877	0.433694	0.447268	0.511382	0.555119	0.664946	0.539321	0.468327	0.455805	0.654341	0.701230	0.670452	0.749824	0.650389	0.533744	0.557373	0.590014	0.583402	0.729517	0.828638	0.585383	0.625468	0.593743	0.480860	0.513310	0.595156	0.655568
30486	FOODS_3_824_WI_3_evaluation	0.259211	0.254881	0.238968	0.225033	0.226900	0.287327	0.297278	0.241045	0.242830	0.271071	0.368390	0.381267	0.376315	0.420627	0.360178	0.316844	0.361534	0.345815	0.258029	0.379033	0.406634	0.289211	0.344375	0.349159	0.273884	0.241947	0.298811	0.298772
30487	FOODS_3_825_WI_3_evaluation	0.630791	0.529853	0.491117	0.477037	0.544271	0.641009	0.701201	0.662344	0.525250	0.674044	1.036723	1.112819	0.895466	1.287720	1.139596	0.811612	0.999034	0.964596	0.775849	1.146324	1.273331	0.942830	1.058475	1.068908	0.743412	0.696659	0.798183	0.936363
30488	FOODS_3_826_WI_3_evaluation	1.014185	1.014675	0.968098	0.941371	1.094441	1.173944	1.154782	1.163941	1.041478	0.993564	1.318697	1.492611	1.347546	1.528297	1.231180	1.068935	1.204987	1.188645	1.066231	1.481773	1.546942	1.180402	1.334267	1.270570	1.031520	1.117360	1.197175	1.297182
30489	FOODS_3_827_WI_3_evaluation	1.808649	1.595394	1.546023	1.589576	1.912314	1.996711	1.802233	1.733731	1.550154	1.312341	1.744125	2.024802	1.988244	1.903606	1.651901	1.457500	1.466568	1.542802	1.580044	1.940835	1.915329	1.624481	1.829056	1.647234	1.447651	1.640128	1.813909	1.841560
30490 rows × 29 columns

In [ ]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL + 'sample_submission.csv')[['id']]
submission = submission.merge(final_all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission'+'.csv', index=False)

In [12]:
!cp 'submission.csv' /content/gdrive/'My Drive'/m5_new